In [1]:
import os
import operator
import gc
import pandas as pd
from tqdm import tqdm
from random import *
import pickle
import csv
import re
from num2words import num2words
from transliterate import translit

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

def save_obj(obj, name):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, -1)

def load_obj(name):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def insert_key(key, arr, k = 1):
    #if hasNumbers(arr): #no_digits
    #    return
    if key not in res_new:
        res_new[key] = {arr : k}
    else:
        if arr in res_new[key]:
            res_new[key][arr] += k
        else:
            res_new[key][arr] = k  
            
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

INPUT_PATH = r'./input'
DATA_INPUT_PATH = r'./input/ru_with_types'
SUBM_PATH = INPUT_PATH

punct = {'«','.','»',','}
dash = {'-','—'}
short = {"по","англ","ее","что","есть","где","кто","две","ибн","ту"}

res_new = dict()

df = pd.read_csv("input/ru_test_2.csv")
print(df.head())
print(df.tail())
test_set_old = set(df.before)
print(len(test_set_old))

   sentence_id  token_id        before
0            0         0           Эта
1            0         1         книга
2            0         2             ,
3            0         3  отличающаяся
4            0         4             «
        sentence_id  token_id     before
989875        69999        17     убедил
989876        69999        18        его
989877        69999        19  выполнить
989878        69999        20     приказ
989879        69999        21          .
175991


In [ ]:
df = pd.read_csv("input/ru_train.csv")
#dp = df[df.before.isin(ss)] 
print(len(dp))

res_new = dict()
for i in tqdm(dp.index):
    ans = " ".join(list(df[df.sentence_id == df.iloc[i].sentence_id].before))
    try:
        res_new[arr[0]].add((arr[1], i, ans))
    except:
        res_new[arr[0]] = {(arr[1], i, ans)}

In [ ]:
#df = pd.read_csv("input/ru_train.csv")
#dp = df[df.before.isin(multi_words)] 

train = open(os.path.join(INPUT_PATH, "ru_train.csv"), encoding='UTF8')
line = train.readline()
res_new = dict()
last = ""
for i in tqdm(range(10574517)):
    line = train.readline().strip()
    if line == '':
        print("Finish")
        break
    
    pos = line.find('","')
    text = line[pos + 2:]
    if text[:3] == '","':
        continue
    text = text[1:-1]
    arr = text.split('","')
    
    if arr[0] not in multi_words:
        continue
    if df.iloc[i].before != arr[0]:
        print("error")
    
    ans = " ".join(list(df[df.sentence_id == df.iloc[i].sentence_id].before))
    try:
        res_new[arr[0]].add((arr[1], i, ans))
    except:
        res_new[arr[0]] = {(arr[1], i, ans)}
        
    #print(res_new)
    #break
    
    #insert_key(arr[0], arr[1])
    
train.close()
gc.collect()
len(res_new)
#800813 801234

In [ ]:
#save_obj(res_new, "res_new_simple_all_no_digits_train") 
res_new = load_obj("res_new_simple_all_no_digits_train") 

In [ ]:
files = os.listdir(DATA_INPUT_PATH)
for file in tqdm(files):
    train = open(os.path.join(DATA_INPUT_PATH, file), encoding='UTF8')
    while 1:
        line = train.readline().strip()
        if line == '':
            break
        
        pos = line.find('\t')
        text = line[pos + 1:]
        if text[:3] == '':
            continue
        arr = text.split('\t')
        if arr[0] == '<eos>':
            continue
        
        if arr[1] == '<self>' or arr[1] == 'sil':
            arr[1] = arr[0]
    
        insert_key(arr[0], arr[1])
        
    train.close()
    gc.collect()
len(res_new)
#4922825

In [ ]:
#save_obj(res_new, "res_new_simple_all_digits") 
res_new = load_obj("res_new_simple_all_digits") 
res = [x for x in res_new if len(res_new[x])==1]
print(len(res))

single = set(res) & test_set_old
print(len(single))

multi_words = list(test_set_old-set(res)-punct-dash-short)
print(len(multi_words))

s = dict()
for x in tqdm(single):
    s[x] = list(res_new[x])[0]
    
save_obj(s, "s_digits")  

# 4832560
# 175716
# 10650
# 4835693
# 175442
# 10924

In [ ]:
test_set = set()

df = pd.read_csv("input/ru_test_2.csv")
dp = df[df.before.isin(multi_words)] 

before = df.before
token = df.token_id

for i in tqdm(dp.index):
    arr = before[i]
    
    try:
        if token[i-1] == 0:
            lost = ""
        else:
            lost = before[i-2]
    except:
        lost = ""
        
    try:
        if token[i] == 0:
            last = ""
        else:
            last = before[i-1]
    except:
        last = ""
    
    if last=="":
        lost=""
        
    try:
        if token[i+1]==0:
            nex = ""
        else:
            nex = before[i+1]
    except:
        nex = ""
        
    try:
        if token[i+2]==0:
            nexx = ""
        else:
            nexx = before[i+2]
    except:
        nexx = ""
    if nex=="":
        nexx=""
    
    for key in [(lost, last, arr, nex, nexx),
                
                (lost, last, arr, nex, None),
                (None, last, arr, nex, nexx),
                
                (lost, last, arr, None, None),
                (None, last, arr, nex, None),
                (None, None, arr, nex, nexx),
                
                (None, last, arr, None, None), 
                (None, None, arr, nex, None),
                
                (None, None, arr, None, None)]:
        test_set.add(key)
len(test_set)
#125432
#243316
#428862

In [ ]:
save_obj(test_set, "test_set_2_new_digits_x")  

In [45]:
test_set = load_obj("test_set_2_new_digits_x")
#print(len(test_set))

res_new = dict()
big_set = set()
big_dict = dict()

df = pd.read_csv("input/ru_train.csv")
#dp = df[df.before.isin(multi_words)] 
dp = df[df.before.isin(ss)] 

before = df.before
after = df.after
token = df.token_id

for i in tqdm(dp.index):
    arr = [before[i], after[i]]
    
    try:
        if token[i-1] == 0:
            lost = ""
        else:
            lost = before[i-2]
    except:
        lost = ""
        
    try:
        if token[i] == 0:
            last = ""
        else:
            last = before[i-1]
    except:
        last = ""
    
    if last=="":
        lost=""
        
    try:
        if token[i+1]==0:
            nex = ""
        else:
            nex = before[i+1]
    except:
        nex = ""
        
    try:
        if token[i+2]==0:
            nexx = ""
        else:
            nexx = before[i+2]
    except:
        nexx = ""
    if nex=="":
        nexx=""
    
    if ((lost, last, arr[0], nex, nexx) in sss):
        
        ans = "___".join(list(df[df.sentence_id == df.iloc[i].sentence_id].before))
        t = df.iloc[i].token_id
        try:
            res_new[arr[0]].add((arr[1], t, ans))
        except:
            res_new[arr[0]] = {(arr[1], t, ans)}
        
    
#     for key in [(lost, last, arr[0], nex, nexx),
                
#                 (lost, last, arr[0], nex, None),
#                 (None, last, arr[0], nex, nexx),
                
#                 (lost, last, arr[0], None, None),
#                 (None, last, arr[0], nex, None),
#                 (None, None, arr[0], nex, nexx),
                
#                 (None, last, arr[0], None, None), 
#                 (None, None, arr[0], nex, None),
                
#                 (None, None, arr[0], None, None)]:
#         big_set.add(key)
#         try:
#             big_dict[(key, arr[1])]+= 1
#         except:
#             big_dict[(key, arr[1])] = 1
            
# inter = big_set & test_set
# print(len(inter))
# for key, arr in tqdm(list(big_dict)):
#     if key in inter:
#         insert_key(key, arr, big_dict[(key, arr)])

len(res_new)
#125432
#62275
#243316
#97811
#428862
#138359

100%|██████████| 311298/311298 [01:21<00:00, 3802.28it/s]


94

In [46]:
save_obj(res_new, "res_new_2_cheat")  

In [ ]:
save_obj(res_new, "res_new_2")  

In [51]:
test_set = load_obj("test_set_2_new_digits_x")
#print(len(test_set))

res_new = load_obj("res_new_2_cheat")  
#res_new = load_obj("res_new_2")  

files = os.listdir(DATA_INPUT_PATH)
for file in tqdm(files):
    df = pd.read_csv(DATA_INPUT_PATH+"/"+file, sep="\t", low_memory = False, error_bad_lines = False, quoting=3)
    df.columns = [0,1,2]
    
    #dp = df[df[1].isin(multi_words)] 
    dp = df[df[1].isin(ss)] 
    
    before = df[1]
    after = df[2]
    
    big_set = set()
    big_dict = dict()
    
    for i in dp.index:
        arr = [before[i], after[i]]
        if arr[1] == '<self>' or arr[1] == 'sil':
            arr[1] = arr[0]
        
        try:
            if before[i-2]=="<eos>":
                lost = ""
            else:
                lost = before[i-2]
        except:
            lost = ""
        
        try:
            if before[i-1]=="<eos>":
                last = ""
            else:
                last = before[i-1]
        except:
            last = ""
            
        if last=="":
            lost=""
            
        try:
            if before[i+1]=="<eos>":
                nex = ""
            else:
                nex = before[i+1]
        except:
            nex = ""
        try:
            if before[i+2]=="<eos>":
                nexx = ""
            else:
                nexx = before[i+2]
        except:
            nexx = ""
        if nex=="":
            nexx=""
            
        if ((lost, last, arr[0], nex, nexx) in sss):
            ans = before[i]
            t = 0
            j=i-1
            while before[j] != "<eos>":
                ans = before[j] + "___" + ans
                t+=1
                j-=1
                if j==-1:
                    break
            
            j=i+1
            while before[j] != "<eos>":
                ans +=  "___" + before[j]
                j+=1                 
                
            try:
                res_new[arr[0]].add((arr[1], t, ans))
            except:
                res_new[arr[0]] = {(arr[1], t, ans)}

        
#         for key in [(lost, last, arr[0], nex, nexx),
                
#                     (lost, last, arr[0], nex, None),
#                     (None, last, arr[0], nex, nexx),

#                     (lost, last, arr[0], None, None),
#                     (None, last, arr[0], nex, None),
#                     (None, None, arr[0], nex, nexx),

#                     (None, last, arr[0], None, None), 
#                     (None, None, arr[0], nex, None),

#                     (None, None, arr[0], None, None)]:
            
#             big_set.add(key)
#             try:
#                 big_dict[(key, arr[1])]+= 1
#             except:
#                 big_dict[(key, arr[1])] = 1
    
#     inter = big_set & test_set
#     for key, arr in list(big_dict):
#         if key in inter:
#             insert_key(key, arr, big_dict[(key, arr)])
    print(len(res_new))



#243316
#236993
#428862
#424731
#428823


  0%|          | 0/100 [00:00<?, ?it/s]

462148



  1%|          | 1/100 [00:22<36:34, 22.17s/it]

94


  2%|▏         | 2/100 [00:41<34:46, 21.29s/it]

...

135


 99%|█████████▉| 99/100 [31:13<00:18, 18.73s/it]

135


100%|██████████| 100/100 [31:33<00:00, 19.02s/it]

135


In [52]:
save_obj(res_new, "res_upd_2_cheat")  

In [ ]:
save_obj(res_new, "res_upd_2")  

In [70]:
for x in res_new['2009']:
    print(x[0])

двумя тысячами девятью


In [295]:
res_new  = load_obj("res_new_2_cheat") 
res_upd  = load_obj("res_upd_2")
s = load_obj("s_digits")
res = [x for x in res_new if len(res_new[x])==1]

ch = {"0":"ноль",
      "1":"один",
      "2":"два",
      "3":"три",
      "4":"четыре",
      "5":"пять",
      "6":"шесть",
      "7":"семь",
      "8":"восемь",
      "9":"девять"
     }

SUB = str.maketrans("₀₁₂₃₄₅₆₇₈₉", "0123456789")
SUP = str.maketrans("⁰¹²³⁴⁵⁶⁷⁸⁹", "0123456789")

multi_new = set()
multi_upd = list()
single_set = set()

df = pd.read_csv("input/ru_test_2.csv")

before = df.before
token = df.token_id
sentence = df.sentence_id

ides = list()
after = list()
reason = list()

full = {"проверено","архивировано", "с", "от", "умер", "родился", "первоисточника"}
space = {"на","по"} 

m = {}
m['км²'] = 'квадратных километров'
m['км2'] = 'квадратных километров'
m['km²'] = 'квадратных километров'
m['км'] = 'километрах'
m['km'] = 'километрах'
m['кг'] = 'килограмма'
m['kg'] = 'килограмма'
m['m²'] = 'квадратных метров'
m['м²'] = 'квадратных метров'
m['м³'] = 'кубических метров'

m["млн"] = "миллионов" 
m["м/с"] = "метров в секунду"
m["мм"] = "миллиметров"
m["м"] = "метров"
m["ч"] = "часов"
m["л"] = "лет"
m["тыс"] = "тысяч"
m["тонн"] = "тонн"
m["га"] = "гектара"
m["гг"] = "годы"
m["млрд"] = "миллиардов"
m["км/ч"] = "километров в час"
m["руб"] = "рублей"
m["с"] = "секунд"
m["м3"] = "кубических метров"
m["н. э."] = "нашей эры"
m["трлн"] = "триллионов"
m["$"] = "долларов сэ ш а"
m["€"] = "евро"
m["%"] = "процентов"
m["долл"] = "долларов"
m["ГВт"] = "гигаватт"
m["МВт"] = "мегаватт"
m["кВт"] = "киловатт"
m["куб.см"] = "кубических сантиметров"
m["мкм"] = "микрометров"
m["об/мин"] = "оборотов в минуту"
m["нм"] = "нанометра"



for i in tqdm(range(len(df))):
    arr = before[i]
    i1 = sentence[i]
    i2 = token[i]
    
    ides.append(str(i1) + '_' + str(i2))
    
    try:
        if token[i-1] == 0:
            lost = ""
        else:
            lost = before[i-2]
    except:
        lost = ""
        
    try:
        if token[i] == 0:
            last = ""
        else:
            last = before[i-1]
    except:
        last = ""
    
    if last=="":
        lost=""
        
    try:
        if token[i+1]==0:
            nex = ""
        else:
            nex = before[i+1]
    except:
        nex = ""
        
    try:
        if token[i+2]==0:
            nexx = ""
        else:
            nexx = before[i+2]
    except:
        nexx = ""
    if nex=="":
        nexx=""
    
    if arr in s: 
        tmp = s[arr]
        q=0
    elif arr in punct or arr in short:
        tmp = arr
        q=1
    elif arr in dash:
        if before[i-2] == "от" and lost!="":
            tmp = "до"
            q=3
        else:
            tmp = arr
            q=2
#     elif str(arr) == "nan":
#         tmp = "н_trans а_trans н_trans" #"n a" 
#         q = 0              
#     elif full_date(arr)!="" and last in full:     
#         q = -57
#         tmp = full_date(arr)
    else:
        found = False
        q = 1
        for key in [(lost, last, arr, nex, nexx),
                
                    (lost, last, arr, nex, None),
                    (None, last, arr, nex, nexx),

                    (lost, last, arr, None, None),
                    (None, last, arr, nex, None),
                    (None, None, arr, nex, nexx),

                    (None, last, arr, None, None), 
                    (None, None, arr, nex, None),

                    (None, None, arr, None, None)
                   ]:

            if key in res_upd:
                srtd = sorted(res_upd[key].items(), key=operator.itemgetter(1), reverse=True)
                if len(srtd)>1:
                    if arr in res_new:
                        ans = "___".join(list(df[df.sentence_id == df.iloc[i].sentence_id].before))
                        for x in res_new[arr]:
                            if x[2]==ans:
                                found = True
                                q = -100
                                tmp = x[0]
                                break                    
                    
                    tmp = space_date(arr)
                    if tmp !="" and last.lower() in space|{",","—",'('}:
                        found = True
                        q = -22
                        break
                                        
                    tmp = full_date(arr)
                    if tmp !="" and last.lower() in full:
                        found = True
                        q = -21
                        break
                    
                tmp = srtd[0][0]
                found = True
                q*=10
                                
                break
            q+=1
        if not found:
            if str(arr) == "nan":
                tmp = "н_trans а_trans н_trans" #"n a" 
                q = 0
            else:
                if last.lower() in space|{",","—",'('}:
                    tmp = space_date(arr)
                    q = -1.2
                else:    
                    tmp = full_date(arr)
                    q = -1.1
                if tmp=="":
                    ar = arr.replace(" г.", '').replace(" года", '')
                    tmp = dot_date(ar)
                    q=-10
                    if tmp=="":
                        ####  CHECK 10
                        if len(arr)>10 and arr.isnumeric():
                            tmp = " ".join([ch[x] for x in arr])
                            q=-7
                        else: 
                            if arr[0] == "$":
                                arr = arr[1:] + " $"
                            l = []
                            
                            #word = word.replace("(","").replace(")","")
                            
                            spl = arr.split(" ")
                            if len(spl)>1:
                                ar = ''
                                for i in range(len(spl)):
                                    ar += spl[i]
                                    if i+1 < len(spl):
                                        if not spl[i].isnumeric() or not spl[i+1].isnumeric():
                                            ar += " "
                                arr = ar
                            
                            for word in arr.split(" "):
                                key = (None, None, word, None, None)
                                if key in res_upd:
                                    srtd = sorted(res_upd[key].items(), key=operator.itemgetter(1), reverse=True)
                                    t = srtd[0][0]
                                    q = -2
                                    if len(srtd)>1:
                                        p = space_date(word)
                                        if p !="" and last.lower() in space|{",","—",'('}:
                                            q = -24
                                            t = p

                                        p = full_date(arr)
                                        if p !="" and last.lower() in full:
                                            q = -23
                                            t = p
                                    
                                    l.append(t)
                                else:
                                    if word in m:
                                        l.append(m[word])
                                    elif word.replace(".","") in m:
                                        l.append(m[word.replace(".","")])
                                    elif is_ascii(word):
                                        if word.isalpha():
                                            t = trans(word)
                                            l.append(t)
                                            q = -3
                                        else:
                                            try:
                                                l.append(num2words(word, lang='ru'))
                                                q = -4
                                            except:
                                                try:
                                                    l.append(num2words(word.translate(SUB).translate(SUP), lang='ru'))
                                                    q = -4
                                                except:
                                                    if (word.count("-")>0 and word.count(".")==0 
                                                        and word.count("/")==0 and word.count("(")==0
                                                        and word.count("B")==0):

                                                        l.append(dash_num(word.replace(" ","-")).replace("одна ",""))
                                                        q=-50
                                                    else:
                                                        l.append(word)
                                                        q = -5

                                    else:
                                        try:
                                            l.append(num2words(word, lang='ru'))
                                            q = -4
                                        except:
                                            l.append(word)
                                            q = -6
                            tmp = " ".join(l) 
                            single_set.add(((lost, last, arr, nex, nexx),i))

    if "_letter_latin" in tmp:
        t = tmp
        tmp = tmp.replace("_letter_latin", "_latin")
        q+=1000
        multi_new.add((t,tmp))
    #???
    if tmp[0] == " ":
        tmp = tmp[1:]
    
    if q == -5:
        x = arr
        t = url(arr)
        if t!="":
            tmp = t
            q = -13
    
    after.append(tmp)
    reason.append(q)
    
dic = {"id": ides, "after": after, "type": reason}
dr = pd.DataFrame(data=dic)
dr.to_csv("baseline_ext_rus_my.csv", sep=",", index=False, quoting=csv.QUOTE_ALL, columns=["id","after"])

print(len(multi_new),len(multi_upd), len(single_set))
dr.type.value_counts()

 82%|████████▏ | 814528/989880 [03:02<00:53, 3284.03it/s]

0201
0201


 85%|████████▌ | 843506/989880 [03:09<00:33, 4342.64it/s]

701
701
701


 91%|█████████ | 897991/989880 [03:23<00:38, 2379.31it/s]

676
676


100%|██████████| 989880/989880 [03:46<00:00, 4372.95it/s]


18 0 11951


 0.0       766756
 1.0       117085
 2.0        31352
 10.0       23772
 20.0        9538
-6.0         7011
 40.0        5760
 30.0        5013
-3.0         3319
 60.0        2645
 50.0        2445
-1.1         2223
 3.0         1995
 70.0        1979
-21.0        1369
 80.0         991
 90.0         947
 1010.0       851
-4.0          556
 1040.0       494
-50.0         467
 1070.0       451
 1020.0       439
-22.0         430
-1.2          309
-5.0          281
-13.0         231
-10.0         222
 1030.0       173
 1060.0       170
 1080.0       168
 1050.0       146
 1090.0       114
-7.0           92
-2.0           86
Name: type, dtype: int64

In [290]:
x

'j.scijus.2013.08.006.David'

In [296]:
dc = pd.read_csv("baseline_ext_rus_my_9926.csv")
dc.head()

dc["new"] = dr["after"]
dc[dc["after"] != dc["new"]]


,id,after,new
16422,1156_2,2016.norwegiancharts.com,две тысячи шестнадцать точка н_trans о_trans р...
36240,2554_4,www.academia.edu/9093152/,w w w точка а_trans к_trans а_trans д_trans е_...
41672,2935_20,j.scijus.2013.08.006.David,й_trans точка с_trans к_trans и_trans ю_trans ...
52788,3746_3,L точка e точка r точка a,л_trans точка точка р_trans точка а_trans
61509,4370_17,11.sejums,одиннадцать точка с_trans е_trans ю_trans м_tr...
66973,4765_14,a точка h точка M точка S,а_trans точка х_trans точка м_trans точка с_trans
70552,5018_24,точка д_trans о_trans к_trans,точка д_trans о_trans к_trans
72030,5128_2,2016.pythonOCC,две тысячи шестнадцать точка п_trans и_trans т...
75218,5354_5,1086/383486,одна тысяча восемьдесят шесть триста восемьдес...
75293,5361_5,r точка t точка a,р_trans точка т_trans точка а_trans


In [ ]:
a = 0
for x in multi_upd:
    if x[0][1]=="(":
        t =space_date(x[0][2]) 
        if t!="" and t!=x[1][0]:
            print(x[0], t) #, )
            a+=1
a

In [3]:
import nltk
from functools import lru_cache
from itertools import product as iterprod

try:
    arpabet = nltk.corpus.cmudict.dict()
except LookupError:
    nltk.download('cmudict')
    arpabet = nltk.corpus.cmudict.dict()

@lru_cache()
def wordbreak(s):
    s = s.lower()
    if s in arpabet:
        return arpabet[s]
    middle = len(s)/2
    partition = sorted(list(range(len(s))), key=lambda x: (x-middle)**2-x)
    for i in partition:
        pre, suf = (s[:i], s[i:])
        if pre in arpabet and wordbreak(suf) is not None:
            return [x+y for x,y in iterprod(arpabet[pre], wordbreak(suf))]
    return None

def wordbreak2(s):
    s = s.lower()
    if s in arpabet:
        return arpabet[s][0]
    for i in range(len(s)-1,0,-1):
        if s[:i] in arpabet:
            #print(i, s[:i],s[i:],arpabet[s[:i]][0],wordbreak2(s[i:])[0])
            return arpabet[s[:i]][0] + wordbreak2(s[i:])

wordbreak2("iPad")

['IH1', 'P', 'AE1', 'D']

In [292]:
def trans(x):
    if len(x)>1:
        if x in s:
            return s[x]
        
        key = (None, None, x, None, None)
        if key in res_upd:
            srtd = sorted(res_upd[key].items(), key=operator.itemgetter(1), reverse=True)
            t = srtd[0][0]
            return t
    
    word = x.lower()
    t = ("_trans ".join(translit(word, 'ru')) + "_trans")
    t = t.replace("w_trans","в_trans")
    t = t.replace("ы_trans","и_trans")
    t = t.replace("щ_trans","ш_trans")
    t = t.replace("ц_trans","к_trans")
    t = t.replace("а_trans л_trans л_trans","о_trans л_trans")
    t = t.replace("т_trans и_trans о_trans","ш_trans е_trans")
    t = t.replace("и_trans е_trans в_trans","ь_trans ю_trans")
    t = t.replace("т_trans у_trans р_trans","ч_trans е_trans р_trans")
    t = t.replace("е_trans в_trans","ь_trans ю_trans")
    t = t.replace("а_trans и_trans","э_trans й_trans")
    t = t.replace("г_trans у_trans","г_trans а_trans")
    t = t.replace("о_trans и_trans","о_trans й_trans")
    t = t.replace("т_trans х_trans","т_trans")
    t = t.replace("о_trans о_trans","у_trans")
    t = t.replace("е_trans е_trans","и_trans")
    t = t.replace("п_trans х_trans","ф_trans")
    t = t.replace("с_trans ч_trans","ш_trans")
    t = t.replace("а_trans у_trans","о_trans")
    t = t.replace("о_trans а_trans","о_trans")
    t = t.replace("в_trans х_trans","у_trans")
    t = t.replace("к_trans к_trans","к_trans")
    t = t.replace("х_trans н_trans","н_trans")
    t = t.replace("х_trans л_trans","л_trans")
    t = t.replace("х_trans р_trans","р_trans")
    t = t.replace("х_trans к_trans","к_trans")
    t = t.replace("x_trans","к_trans с_trans")
    t = t.replace("х_trans у_trans","х_trans а_trans")
    t = t.replace("е_trans и_trans","е_trans й_trans")
    t = t.replace("с_trans у_trans","с_trans а_trans")
    t = t.replace("п_trans у_trans","п_trans а_trans")
    t = t.replace("г_trans и_trans я_trans","д_trans ж_trans и_trans я_trans")
    t = t.replace("у_trans н_trans т_trans","а_trans н_trans т_trans")
    t = t.replace("у_trans н_trans д_trans","а_trans н_trans д_trans")
    t = t.replace("у_trans н_trans к_trans","а_trans н_trans к_trans")
    t = t.replace("у_trans б_trans","а_trans б_trans")
    t = t.replace("с_trans в_trans","с_trans у_trans")
    #t = t.replace("к_trans у_trans","с_trans у_trans")

    if t.endswith("е_trans"):
        t = t[:-8]

    if t.endswith("г_trans у_trans"):
        t = t[:-8]

    if t.startswith("е_trans"):
        t = "э" + t[1:]

    if word[0] == "w" and len(word)>1:
        t = "у" + t[1:]

    if word[0] == "j" and len(word)>1:
        tt = "д_trans ж_trans"
        if word[1]=="u":
            tt+=" y_trans"
        if word[1]=="a":
            tt+=" е_trans"
        t = tt + t[7:]

    #if word.startswith("wi"):
    #    t = "в" + t[1:]

    if word.endswith("ts") and t.endswith("к_trans"):
        t = t[:-8] + " т_trans с_trans"

    if word.endswith("ux"):
        t = t[:-16]

    if word.endswith("ge") and t.endswith("г_trans"):
        t = t[:-8] + " д_trans ж_trans"

    if t.startswith("а_trans и_trans р_trans"):
        t = "а_trans й_trans" + t[15:]

    if t.endswith("и_trans а_trans"):
        t = t[:-8] + " я_trans"

    if t.endswith("е_trans с_trans") and not word.endswith("ses") and not word.endswith("hes"):
        t = t[:-15] + "с_trans"

    if t.endswith("д_trans г_trans"):
        t = t[:-8] + " ж_trans"

    if len(word)>2 and word[0]=='c' and word[1]=='h' and word[2] not in gl:
        t = "к" + t[1:]

    if "ce" in word:
        t = t.replace("к_trans","с_trans") 

    t = t.replace("у_trans н_trans д_trans","а_trans н_trans д_trans")
    
    return t

trans("fuck")

'ф_trans а_trans к_trans'

In [294]:
def url(x):
    s = x.split("/")
    if len(s) == 2 and s[0].isnumeric() and s[1].isnumeric():
        res = num2words(s[0], lang='ru') + " " + num2words(s[1], lang='ru')
        if last:
            if res.endswith("один"):
                res = res.replace("один","первых")
            elif res.endswith("два"):
                res = res.replace("два","вторых")
            elif res.endswith("три"):
                res = res.replace("три","третьих")
            elif res.endswith("четыре"):
                res = res.replace("четыре","четвертых")
            elif res.endswith("семь"):
                res = res.replace("семь","седьмых")
            elif res.endswith("восемь"):
                res = res.replace("восемь","восьмых")
            elif res.endswith("сто"):
                res = res.replace("сто","сотых")
            else:
                res = res[:-1] + "ых"
            return res

    res = ""
    if x.startswith("http://www."):
        res = "h t t p w w w точка "
        x = x.replace("http://www.", "")
    elif x.startswith("http://"):
        res = "h t t p "
        x = x.replace("http://", "")
    
    ss = x.split("/")
    #print(ss)
    i = 0
    for z in ss:
        if z == "":
            continue
        s = z.split(".")
        #print(s)
        for y in s:
            if y =="":
                continue
            if y.isnumeric():
                res += num2words(y, lang='ru') + " точка "
            elif not y.isalpha() or not is_ascii(y):
                #print(y)
                return ""
            else:
                res += trans(y) + " точка " 
        res = res[:-7] + " косая черта "
    if x[-1] == "/":
        res += " косая черта"
    res = res[:-13]
    return res
    
url("http://2017.fuck.com/te.st/w/")
#url("100/101")
    

'h t t p две тысячи семнадцать точка ф_trans а_trans к_trans точка к_trans о_trans м_trans косая черта т_trans точка с_trans т_trans косая черта в_trans косая черта'

In [223]:
spl = "19 779 га".split(" ")
if len(spl)>1:
    ar = ''
    for i in range(len(spl)):
        ar += spl[i]
        if i+1 < len(spl):
            if not spl[i].isnumeric() or not spl[i+1].isnumeric():
                ar += " "
    arr = ar
arr

'19779 га'

In [271]:
num = {"0","1","2","3","4","5","6","7","8","9"}
alp = {"q","w","e","r","t","y","u","i","o","p",
       "a","s","d","f","g","h","j","k","l",
       "z","x","c","v","b","n","m"}

def sepup(x):
    return " ".join([c.lower() for c in x.replace(".","").replace(" ","")])

k = -5
a = 0
z = set()
print(len(dr[dr.type==k]))
dr[dr.type==k]
for x in dr[dr.type==k]["after"]:
#for x in df[dr.type==k]["before"]:
    #if x.endswith(" г.") or x.endswith(" года"):
    #if (len(set(x.lower())&alp)==0 and len(set(x.lower())&num)!=0):
    #if "/" in x:
    #if x[:2] == "20":
    #if "a" in x and "а" in x:
    print(x)
    a+=1
    #break
a


281
(PS2
Its-behind-you.com
29.tv21
http://www.sportskeeda.com/football/tony-pulis-one-of-the-best-in-the-epl
II)
(NS
http://www.geonames.org/347585The
http://udarenie.myreutov.ru/dictionaries/geo_mo_slovar_2007
(II)
(II
(NaN)
PS3
Game-Interviews.com
H1N1
1:19.38
docs.cntd.ru/document/1200115397Lighting
SH2
SH2
SH2
II)
SF9010
51,89%
II)
PS4
KH2
(II)
(PS3
https://www.ibccrim.org.br/noticia/13346-Revogada-a-contravencao-penal-de-mendicancia)
M. D. W.
(II)
SP286
5.hidemyass.com/ip-1/encoded/Oi8vdHJ1Y2tzLmRtaXIucnUvd2lraS90b25hci8%253D
II)
HF)
IP7
zee-tv.ru
fromhttp://www.sciencedaily.com/releases/2006/11/061130081347.htmShannon
game-exe.ru
reporter63.ru
(SB0
II)
CS161
KP40
PS3
451,611,866,957
(II)
http://www.moskva.fm/artist/thomas_anders)
cyberleninka.ru/article/n/detelnost-yu-v-andropova-v-apparate-tsk-kpss-vneshnepoliticheskie-aspekty-1957-1967-gg.pdf
http://usinsk.online/news/23170/https://www.bnkomi.ru/data/news/56255/
http://www.nytimes.com/2015/12/20/arts/music/kurt-masur-new-york-

281

In [288]:
#dq = pd.read_csv("input/ru_train.csv")

a = 0
b = 0
for i in range(500000):
    word = dq.before[i]
    if str(word) == "nan":
        continue
#     if (len(set(word.lower())&alp)==0 and len(set(word.lower())&num)==0
#         and dq.before[i]!=dq.after[i]
#         and word.lower()!=word 
#         and word.upper()!=word
#        ):
    if word == "a":
    #if "долл." in word:
        #if sepup(dq.before[i]) != dq.after[i]: 
        print(dq.before[i], dq.after[i])
        #break
        
#a,b,b/a 
    

a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin
a a_latin


KeyboardInterrupt: 

In [ ]:
a = 0
b = 0
for i in range(800000):
    word = dq.before[i]
    if str(word)=="nan":
        continue
    if dot_date(word)!="":
        continue
    sword = word.split("-")
    if (word.count("-")>0 and word.count(".")==0 and word != "-" and sword[1].isnumeric() and sword[0].isnumeric()
        and word.count("/")==0 and word.count("(")==0
        and word.count("B")==0):
        z = dash_num(word.replace(" ","-")).replace("одна ","")
        a+=1
        if z!=dq.after[i]:
            print(dq.before[i])
            print(dq.after[i])
            print(z,'\n')
            b+=1
a,b,b/a  
    
            
        

In [5]:
def name_num(t):
    if len(t) == 0:
        return ""
    while t[0]=="0":
        return "ноль " + name_num(t[1:])
        
    if len(t) == 4:
        if t[:2] == "20" and t[2]!="0":
            return "двадцать " + num2words(t[2:], lang='ru') + " "
        if t[1]=="0" or t[2]=="0":
            return num2words(t, lang='ru') + " "
        return num2words(t[:2], lang='ru') + " " + num2words(t[2:], lang='ru') + " "
    if len(t) <= 3:
        num2words(t, lang='ru')
    return num2words(t[:3], lang='ru') + " " + name_num(t[3:])

def dash_num(t):
    res = ""
    s = t.split("-")
    for x in s:
        res += name_num(x) + "sil "
    return res[:-5]

dash_num("978-5-104935-25-2")
#name_num("12")


'девятьсот семьдесят восемь sil пять sil сто четыре девятьсот тридцать пять sil двадцать пять sil два'

In [14]:
mm = {"1":"января",
      "2":"февраля",
      "3":"марта",
      "4":"апреля",
      "5":"мая",
      "6":"июня",
      "7":"июля",
      "8":"августа",
      "9":"сентября",
      "10":"октября",
      "11":"ноября",
      "12":"декабря",
      "01":"января",
      "02":"февраля",
      "03":"марта",
      "04":"апреля",
      "05":"мая",
      "06":"июня",
      "07":"июля",
      "08":"августа",
      "09":"сентября"      
     }

d = {"0":"",
     "1":"первого",
     "2":"второго",
     "3":"третьего",
     "4":"четвертого",
     "5":"пятого",
     "6":"шестого",
     "7":"седьмого",
     "8":"восьмого",
     "9":"девятого",
     "01":"первого",
     "02":"второго",
     "03":"третьего",
     "04":"четвертого",
     "05":"пятого",
     "06":"шестого",
     "07":"седьмого",
     "08":"восьмого",
     "09":"девятого",
     "10":"десятого",
     "11":"одиннадцатого",
     "12":"двенадцатого",
     "13":"тринадцатого",
     "14":"четырнадцатого",
     "15":"пятнадцатого",
     "16":"шестнадцатого",
     "17":"семнадцатого",
     "18":"восемнадцатого",
     "19":"девятнадцатого",
     "20":"двадцатого",
     "21":"двадцать первого",
     "22":"двадцать второго",
     "23":"двадцать третьего",
     "24":"двадцать четвертого",
     "25":"двадцать пятого",
     "26":"двадцать шестого",
     "27":"двадцать седьмого",
     "28":"двадцать восьмого",
     "29":"двадцать девятого",
     "30":"тридцатого",
     "31":"тридцать первого",
    }

dd = {"2":"двадцать",
      "3":"тридцать",
      "4":"сорок",
      "5":"пятьдесят",
      "6":"шестьдесят",
      "7":"семьдесят",
      "8":"восемьдесят",
      "9":"девяносто"
     }

dd0 = {"1":"десятого",
       "2":"двадцатого",
       "3":"тридцатого",
       "4":"сорокового",
       "5":"пятидесятого",
       "6":"шестидесятого",
       "7":"семидесятого",
       "8":"восьмидесятого",
       "9":"девяностого"
     }

ddd = {"1":"сто",
       "2":"двести",
       "3":"триста",
       "4":"четыреста",
       "5":"пятьсот",
       "6":"шестьсот",
       "7":"семьсот",
       "8":"восемьсот",
       "9":"девятьсот"
     }

gl = ["a","e","u","i","o","y"]
zv = [ "b" , "v" , "g" , "d" , "z" , "l" , "m" , "n" , "r" ]

def thous(x):
    if x[0]=="1":
        res = "тысяча "
    if x[0]=="2":
        res = "две тысячи "
    
    if int(x[1])!=0:
        res += ddd[x[1]]+" "
    if int(x[2])>1:
        if x[3]=="0":
            res += dd0[x[2]] 
        else:
            res += dd[x[2]]+" "+d[x[3]]
    else:
        if x[2]=="0":
            res += d[x[3]]
        else:
            if x[3]=="0":
                res += dd0[x[2]]
            else:
                res += d[x[2]+x[3]]
        
    return res + " года"

#print(thous("2007"))

def year(q):
    res = ""
    
    for x in res_upd:
        if x[2].endswith(q):
            try:
                y = x[2].split()[-3]
                z = list(res_upd[x])[0]
                res = z.split(y)[-1]
                break
            except:
                pass
        if q == x[2]:
            try:
                res = " "+list(res_upd[x])[0]
                break
            except:
                pass
    if res == "" and int(q[0])<3 and q[3]!="":
        try:
            res = thous(q[:4])
        except:
            pass
    return res

def full_date(t):
    a = {'г','г.','гг.'}
    b = {'год','года','году'}
    ans = ""
    s = t.split()
    if len(s) == 3 and len(s[-1])==4 and s[-1].isnumeric():
        if s[0] in d:
            res = year(s[2] + " года")
            tmp = d[s[0]]+ " " + s[1] + res
            if res != "":
                ans = tmp
            else:
                print(s[2])
    if len(s) == 4 and s[-1] in a|b:
        if s[0] in d:
            res = year(s[2] + " " + s[-1])
            tmp = d[s[0]]+ " " + s[1] + res
            if res != "":
                ans = tmp
            else:
                print(s[2])
    return ans

def space_date(t):
    t = t.replace(" года",'')
    ans = ""
    s = t.split()
    if len(s) == 3 and len(s[-1])==4 and s[2].isnumeric() and s[0].isnumeric() and not s[1].isnumeric():
        try:
            ans = (d[s[0]][:-2]+"е ").replace("ее",'е') +  s[1] + " " + thous(s[2])
        except:
            pass
    return ans

def dot_date(t):
    res = ""
    s = t.split(".")
    if len(s)!=3:
        s = t.split("-")
        if len(s)!=3:
            s = t.split("/")
            if len(s)!=3:
                return ""
    
    if len(s[2]) == 2:
        s[2] = "19"+s[2]
    if len(s[2])!=4:
        return ""
        
    if  len(s)==3 and s[0].isnumeric() and s[1].isnumeric() and s[2].isnumeric():
        try:
            res = (d[s[0]][:-2]+"е ").replace("ее",'е') +  mm[s[1]] + " " + thous(s[2])
        except:
            pass
        
    return res

#print(dot_date("02/10/15 г.".replace(" г.",'')))
#space_date("11 января 2017 года")
full_date("19 мая 2016 г.")

'девятнадцатого мая две тысячи шестнадцатого года'

In [ ]:
t = "1774"
for x in res_upd:
    if x[2].endswith(t):
        try:
            y = x[2].split()[-3]
            z = list(res_upd[x])[0]
            print(x, y, z.split(y)[-1])
            break
        except:
            pass
    if t == x[2]:
        try:
            z = list(res_upd[x])[0]
            print(x, z)
            break
        except:
            pass
    if t in x[2]:
        try:
            print(x, res_upd[x])
            break
        except:
            pass
    

In [ ]:
cheat = set()
for i in tqdm(range(70000)):
    ans = " ".join([str(x) for x in list(df[df.sentence_id == i].before)])
    cheat.add(ans)
len(cheat)

In [ ]:
check = set()
k = 0
files = os.listdir(DATA_INPUT_PATH)
for file in tqdm(files):
    train = open(os.path.join(DATA_INPUT_PATH, file), encoding='UTF8')
    ans = ""
    while 1:
        line = train.readline().strip()
        if line == '':
            break
        
        pos = line.find('\t')
        text = line[pos + 1:]
        if text[:3] == '':
            continue
        arr = text.split('\t')
        if arr[0] == '<eos>':
            ans = ans[1:]
            if ans in cheat:
                #print(ans)
                k+=1
                check.add(ans)
            ans=""
            continue
        
        ans+= " " + arr[0]
    print(k)
    train.close()
    gc.collect()

In [ ]:
{'13 i 29 Schneider — Kampania Wrzesniowa 1939.pl105 MM ARMATA DALEKONO Ś nan WZ . 13 / WZ . 29105 mm wz .',
 'Am Fam Physician 11 ( 6 ) : 80 ( 9 ) : 1011-1013 . Riviello RJ , Brown nan .',
 'Group nan - nan & RAY CHARLES .',
 'Rasna — самоназвание этрусков ) A - RI - SI - TO - NO - SE A - RA - TO - WA - nan - KA - SO - KO - O - SE KE - RA KE - RE - TU - LO - SEAriston - ose Artowanaksoko - ose , kera keretul - ose .'}

In [ ]:
ss = set()
for x in multi_new:
    ss.add(x[0][2])
    
for x in multi_upd:
    ss.add(x[0][2])
    
len(ss)

In [ ]:
dq = pd.read_csv("input/ru_train.csv")
dq.head(10)

In [ ]:
wordbreak2("Pollienus")[0]

In [ ]:

# 0.4691895512391159
# 0.494962486602358

In [ ]:
rupabet = {
     'AA0':"а",
     'AA1':"а",
     'AA2':"а",
     'AE0':"а",
     'AE1':"а",
     'AE2':"а",
     'AH0':"э",
     'AH1':"а",
     'AH2':"а",
     'AO0':"о",
     'AO1':"о",
     'AO2':"о",
     'AW0':"а_trans у",
     'AW1':"а_trans у",
     'AW2':"а_trans у",
     'AY0':"а_trans й",
     'AY1':"а_trans й",
     'AY2':"а_trans й",
     'B':"б",
     'CH':"ч",
     'D':"д",
     'DH':"з",
     'EH0':"е",
     'EH1':"е",
     'EH2':"е",
     'ER0':"е_trans р",
     'ER1':"е_trans р",
     'ER2':"е_trans р",
     'EY0':"е_trans й",
     'EY1':"е_trans й",
     'EY2':"е_trans й",
     'F':"ф",
     'G':"г",
     'HH':"х",
     'IH0':"и",
     'IH1':"и",
     'IH2':"и",
     'IY0':"и",
     'IY1':"и",
     'IY2':"и",
     'JH':"д_trans ж",
     'K':"к",
     'L':"л",
     'M':"м",
     'N':"н",
     'NG':"н_trans г",
     'OW0':"о",
     'OW1':"о",
     'OW2':"о",
     'OY0':"о_trans й",
     'OY1':"о_trans й",
     'OY2':"о_trans й",
     'P':"п",
     'R':"р",
     'S':"с",
     'SH':"ш",
     'T':"т",
     'TH':"з",
     'UH0':"у",
     'UH1':"у",
     'UH2':"у",
     'UW0':"у",
     'UW1':"у",
     'UW2':"у",
     'V':"в", #ф
     'W':"у",
     'Y':"у", #ь
     'Z':"с",
     'ZH':"ж"}
        

In [ ]:
a = 0
b = 0
x = dict()
y = dict()
z = dict()

full = {"проверено","архивировано", "с", "от", "умер", "родился", "первоисточника"}
space = {"на","по"}

for i in range(2,300000):
    if str(df.before[i])=="nan":
        continue
    s = df.before[i].split()
    if (df.before[i-1].lower() in full and #(119, 1723)
        df.before[i-1].lower() not in space and #space (23, 2)
        df.before[i-1] != "." and #(53, 37) space
        #df.before[i+1] == "," and #(32, 17)
        df.before[i-1] != "," and #(7, 15) full
        df.before[i-1] != ";" and #(18, 18)
        df.before[i-1] != "—" and #(28, 14) #space
        df.before[i-1] != "(" and #(145, 153)
        #df.before[i+2] != "." and #(128, 138)
        len(s) == 3 and len(s[-1])==4 and s[2].isnumeric() and s[0].isnumeric() and not s[1].isnumeric()):
        
        #print(df.before[i-1], df.before[i+1])
        #print(df.before[i], dr.after[i], i)
        if space_date(df.before[i]) != dr.after[i]:
            print(df.before[i-2], df.before[i+2])
            print(df.before[i], dr.after[i], i)
        if full_date(df.before[i]) == dr.after[i]:
            b+=1
        if df.before[i-2] in x:
            x[df.before[i-2]]+=1
        else:
            x[df.before[i-2]] = 1
        if df.before[i+1] in y:
            y[df.before[i+1]]+=1
        else:
            y[df.before[i+1]] = 1
        
        #break
a,b


In [ ]:
df[df["sentence_id"] == df.iloc[19594].sentence_id]

In [ ]:
df.iloc[11937].sentence_id

In [ ]:
!translate en zh-TW <<< 'Hello World!'

In [ ]:
from transliterate import translit, get_available_language_codes
print(" ".join([x+str("_trans") for x in translit("про", 'ru')]))

In [ ]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

for i in dr[df.sentence_id == 5981].index:
    x = dr.iloc[i].after
    if not is_ascii(x) or x in punct|{"(",")","/",":"} or x in dash or "_latin" in x or "_trans" in x:
        t = x
    else:
        t = " ".join([y+str("_trans") for y in translit(df.iloc[i].before.lower(), 'ru')])
    print(i,t,x)
    dr.set_value(i, "after", t)